# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/openair2/spec_ingestor.py F1AP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: gNB-CU crashes when gNB-DU responds to UE Context Modification request with incorrect RNTI in IE.
Repro Steps:
1) The attacker intercepts a legitimate UE Context Modification request.
2) Modify the RNTI in the IE to an incorrect value and forward it to the gNB-CU.
3) For gNB-DUs, the attacker waits for UE release and immediately sends a DLRRCMessage Transfer.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "gNB-CU crashes when gNB-DU responds to UE Context Modification; attacker tampers RNTI IE; DU may also send DL RRC Message Transfer after release.",
    "interface_guess": ["F1AP", "RRC"],
    "procedure_guess": ["UE Context Modification", "DL RRC Message Transfer", "UE Context Release"],
    "components_involved": ["gNB-CU", "gNB-DU"],
    "key_ids": {
      "transaction_id": "",
      "cu_ue_f1ap_id": "",
      "du_ue_f1ap_id": "",
      "du_id": "",
      "rnti": "tampered",
      "pci": "",
      "served_cell_list": []
    },
    "signals_or_timers": [
      "UE CONTEXT MODIFICATION REQUEST",
      "UE CONTEXT MODIFICATION RESPONSE",
      "UE CONTEXT MODIFICATION FAILURE",
      "DL RRC MESSAGE TRANSFER",
      "UE CONTEXT RELEASE COMMAND",
      "UE CONTEXT RELEASE COMPLETE"
    ]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {"Keywords": ["DL RRC", "Message Transfer", "lookup by DU UE ID"], "title": ["8.4.2", "DL RRC Message Transfer"]},
    {"Keywords": ["UE Context Modification", "Failure", "Cause"], "title": ["9.2.2.9", "UE CONTEXT MODIFICATION FAILURE"]},
    {"Keywords": ["UE Context Modification", "Confirm", "Refuse"], "title": ["9.2.2.11", "UE CONTEXT MODIFICATION CONFIRM/REFUSE"]},
    {"Keywords": ["UE Context Release", "Command", "Complete"], "title": ["8.3.3", "UE Context Release (gNB-CU initiated)"]},
    {"Keywords": ["F1AP ID", "UE association", "CU"], "title": ["9.3.1.4", "gNB-CU UE F1AP ID"]},
    {"Keywords": ["F1AP ID", "UE association", "DU"], "title": ["9.3.1.5", "gNB-DU UE F1AP ID"]}
  ],
  "expected_behaviour": "UE-associated F1AP procedures must carry valid gNB-CU and gNB-DU UE F1AP IDs that jointly identify a single UE context. On UE Context Modification, DU may send RESPONSE/FAILURE or REQUIRED; CU must handle FAILURE/REFUSE without aborting. DL RRC MESSAGE TRANSFER shall include the DU UE F1AP ID to retrieve context; if context not retrievable, procedures (including release) are triggered per spec. Any ID mismatch leads to appropriate Cause (e.g., Unknown or inconsistent pair of UE F1AP ID), not a crash."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {"path": "F1AP/f1ap_cu_ue_context_management.c", "function_name": "CU_handle_UE_CONTEXT_MODIFICATION_FAILURE", "reason": "DU may respond with FAILURE; CU handler currently asserts Not implemented (crash risk)."},
    {"path": "F1AP/f1ap_cu_ue_context_management.c", "function_name": "CU_handle_UE_CONTEXT_MODIFICATION_RESPONSE", "reason": "Processes UE Context Modification Response at CU."},
    {"path": "F1AP/f1ap_du_ue_context_management.c", "function_name": "DU_handle_UE_CONTEXT_MODIFICATION_REQUEST", "reason": "Decodes request and triggers DU-side modification logic."},
    {"path": "F1AP/f1ap_cu_rrc_message_transfer.c", "function_name": "CU_send_DL_RRC_MESSAGE_TRANSFER", "reason": "Sends DL RRC MESSAGE TRANSFER; expects valid UE F1AP IDs."},
    {"path": "F1AP/f1ap_du_rrc_message_transfer.c", "function_name": "DU_handle_DL_RRC_MESSAGE_TRANSFER", "reason": "Decodes DL RRC MESSAGE TRANSFER and dispatches to RRC."},
    {"path": "F1AP/f1ap_ids.c", "function_name": "cu_get_f1_ue_data", "reason": "Validates CU<->DU UE F1AP ID binding; used to check consistency."},
    {"path": "F1AP/lib/f1ap_ue_context.c", "function_name": "decode_ue_context_mod_resp", "reason": "Parses UE Context Modification Response IEs including IDs and optional fields (e.g., C-RNTI)."}
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {"kind": "spec", "source": "TS 38.473", "location": "§8.4.2 DL RRC Message Transfer", "text": "The gNB-CU initiates DL RRC MESSAGE TRANSFER. If a UE-associated logical F1-connection exists, the message shall contain the gNB-DU UE F1AP ID IE, which should be used by gNB-DU to lookup the stored UE context. If no UE-associated logical F1-connection exists, it shall be established at reception."},
    {"kind": "spec", "source": "TS 38.473", "location": "§9.2.2.9 UE CONTEXT MODIFICATION FAILURE", "text": "Message sent by gNB-DU to indicate a modification failure. Mandatory IEs: Message Type, gNB-CU UE F1AP ID, gNB-DU UE F1AP ID, Cause. Criticality as per table. Indicates failure without requiring receiver crash."},
    {"kind": "spec", "source": "TS 38.473", "location": "§9.3.1.4 gNB-CU UE F1AP ID", "text": "Uniquely identifies the UE association over F1 within gNB-CU. Type: INTEGER (0..2^32-1)."},
    {"kind": "spec", "source": "TS 38.473", "location": "§9.3.1.5 gNB-DU UE F1AP ID", "text": "Uniquely identifies the UE association over F1 within gNB-DU. Type: INTEGER (0..2^32-1)."},
    {"kind": "spec", "source": "TS 38.473", "location": "§9.3.1.2 Cause", "text": "Includes Radio Network, Transport, Protocol, Misc causes. Radio Network causes include 'Unknown or inconsistent pair of UE F1AP ID' and similar, to report ID-binding errors instead of crashing."},

    {"kind": "code", "source": "F1AP/f1ap_cu_ue_context_management.c", "location": "L170-L173", "text": "int CU_handle_UE_CONTEXT_MODIFICATION_FAILURE(...){ AssertFatal(1 == 0, \"Not implemented yet\\n\"); }"},
    {"kind": "code", "source": "F1AP/f1ap_du_ue_context_management.c", "location": "L127-L135", "text": "if (!decode_ue_context_mod_req(pdu, &req)) { LOG_E(F1AP, \"cannot decode F1 UE Context Modification Request\\n\"); ... } ue_context_modification_request(&req);"},
    {"kind": "code", "source": "F1AP/f1ap_du_rrc_message_transfer.c", "location": "L60-L69", "text": "if (!decode_dl_rrc_message_transfer(pdu, &msg)) { LOG_E(F1AP, \"cannot decode F1 DL RRC message Transfer\\n\"); ... } dl_rrc_message_transfer(&msg);"},
    {"kind": "code", "source": "F1AP/f1ap_cu_rrc_message_transfer.c", "location": "L114-L128", "text": "if (!cu_exists_f1_ue_data(msg.gNB_CU_ue_id)) { LOG_E(F1AP, \"unknown CU UE ID %d\", ...); return -1; } f1_ue_data_t ue_data = cu_get_f1_ue_data(msg.gNB_CU_ue_id); if (ue_data.secondary_ue != msg.gNB_DU_ue_id) { LOG_E(F1AP, \"unexpected DU UE ID ...\"); return -1; }"}
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UE CONTEXT MODIFICATION REQUEST", "precond": "UE F1AP IDs bound (CU<->DU)", "postcond": "DU evaluates and prepares response"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UE CONTEXT MODIFICATION RESPONSE | FAILURE", "precond": "DU processed request", "postcond": "CU updates context or handles failure"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UE CONTEXT MODIFICATION CONFIRM/REFUSE (if REQUIRED path)", "precond": "DU sent REQUIRED", "postcond": "Procedure completes"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UE CONTEXT RELEASE COMMAND (if needed)", "precond": "Context not retrievable / failure", "postcond": "DU releases"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UE CONTEXT RELEASE COMPLETE", "precond": "Release executed", "postcond": "Context removed"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "DL RRC MESSAGE TRANSFER", "precond": "UE context exists (spec: shall include DU UE ID)", "postcond": "DU expects CU to forward to RRC or handle 'not retrievable'"}
  ],
  "state_machine": {
    "states": ["Idle", "UE_Context_Established", "Modification_Requested", "Awaiting_Modification_Result", "Active", "Release_Pending", "Released"],
    "transitions": [
      {"from": "UE_Context_Established", "to": "Modification_Requested", "on": "Send UE CONTEXT MODIFICATION REQUEST"},
      {"from": "Modification_Requested", "to": "Awaiting_Modification_Result", "on": "DU processes"},
      {"from": "Awaiting_Modification_Result", "to": "Active", "on": "Receive RESPONSE", "guard": "F1AP IDs consistent"},
      {"from": "Awaiting_Modification_Result", "to": "UE_Context_Established", "on": "Receive FAILURE", "guard": "Handle per spec (no crash)"},
      {"from": "Active", "to": "Release_Pending", "on": "Trigger Release"},
      {"from": "Release_Pending", "to": "Released", "on": "Receive UE CONTEXT RELEASE COMPLETE"},
      {"from": "Any", "to": "UE_Context_Established", "on": "DL RRC MESSAGE TRANSFER", "guard": "If context exists; else trigger Release/redirect per spec"}
    ]
  },
  "assumptions": [
    "Tampered RNTI manifests as context/ID inconsistency or decode error leading DU to send FAILURE.",
    "CU must accept FAILURE without aborting and may proceed to release if needed.",
    "DL RRC MESSAGE TRANSFER after release should be handled via 'not retrievable' path, not crash."
  ],
  "questions": [
    "Does UE Context Modification (Response/Failure) path in CU perform null checks before accessing fields?",
    "Is C-RNTI present/used in the specific modification IE set exercised here?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {"name": "Message directionality (UE Context Modification)", "result": "PASS", "evidence": ["TS 38.473 §9.2.2.9", "TS 38.473 §9.2.2.11"]},
    {"name": "Req/Resp pairing (Modification Request -> Response/Failure)", "result": "PASS", "evidence": ["TS 38.473 §9.2.2.9", "F1AP/f1ap_cu_ue_context_management.c:CU_handle_UE_CONTEXT_MODIFICATION_RESPONSE"]},
    {"name": "UE F1AP ID binding must identify single UE context", "result": "PASS", "evidence": ["TS 38.473 §9.3.1.4", "TS 38.473 §9.3.1.5", "TS 38.473 §9.3.1.2"]},
    {"name": "Handle UE CONTEXT MODIFICATION FAILURE without crash", "result": "FAIL", "evidence": ["TS 38.473 §9.2.2.9", "F1AP/f1ap_cu_ue_context_management.c:L170-L173 AssertFatal Not implemented"]},
    {"name": "Transaction scoping via Transaction ID", "result": "UNKNOWN", "evidence": ["TS 38.473 §9.3.1.23"]},
    {"name": "DL RRC requires DU UE ID or handle not retrievable", "result": "PASS", "evidence": ["TS 38.473 §8.4.2", "F1AP/f1ap_du_rrc_message_transfer.c:L60-L69"]}
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "Spec defines UE CONTEXT MODIFICATION FAILURE (DU->CU) with mandatory IDs and Cause. CU must handle it gracefully. In OAI, the CU failure handler asserts Not implemented, causing a crash when DU responds with FAILURE (e.g., triggered by malformed/tampered RNTI/ID conditions). Therefore the observed crash is due to missing failure-handling implementation, not a spec issue."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": [
      "TS 38.473 §9.2.2.9",
      "TS 38.473 §9.2.2.11",
      "TS 38.473 §8.4.2",
      "TS 38.473 §9.3.1.4",
      "TS 38.473 §9.3.1.5",
      "TS 38.473 §9.3.1.2"
    ],
    "code_citations": [
      "F1AP/f1ap_cu_ue_context_management.c:L170-L173",
      "F1AP/f1ap_cu_ue_context_management.c:CU_handle_UE_CONTEXT_MODIFICATION_RESPONSE",
      "F1AP/f1ap_cu_rrc_message_transfer.c:L114-L128",
      "F1AP/f1ap_du_rrc_message_transfer.c:L60-L69"
    ],
    "expected_vs_observed": "Expected: CU handles UE CONTEXT MODIFICATION FAILURE per spec using IDs/Cause, optionally proceeding to release or recovery. Observed: DU-side failure (e.g., due to tampered RNTI/ID inconsistency) triggers CU handler that asserts Not implemented, causing a crash. DL RRC MESSAGE TRANSFER is expected to include DU UE ID and be handled even when context transitions occur; crash indicates missing failure-path implementation at CU."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "The F1AP spec defines UE CONTEXT MODIFICATION FAILURE (DU->CU) and requires proper handling using IDs and Cause. In OAI, the CU handler for this message asserts Not implemented, leading to a crash when DU sends FAILURE (e.g., due to tampered RNTI/ID inconsistency). This is a missing failure-path implementation, not a spec violation.",
    "citations": {
      "spec": [
        "TS 38.473 §9.2.2.9",
        "TS 38.473 §9.2.2.11",
        "TS 38.473 §8.4.2",
        "TS 38.473 §9.3.1.4",
        "TS 38.473 §9.3.1.5",
        "TS 38.473 §9.3.1.2"
      ],
      "code": [
        "F1AP/f1ap_cu_ue_context_management.c:L170-L173",
        "F1AP/f1ap_cu_rrc_message_transfer.c:L114-L128",
        "F1AP/f1ap_du_rrc_message_transfer.c:L60-L69"
      ]
    },
    "suggested_repro": [
      "Establish UE context; send UE CONTEXT MODIFICATION REQUEST from CU to DU.",
      "Force DU to reply with UE CONTEXT MODIFICATION FAILURE (e.g., inject mismatched F1AP IDs or malformed/tampered IE).",
      "Observe CU assertion in CU_handle_UE_CONTEXT_MODIFICATION_FAILURE and process termination.",
      "Optionally, after UE release, trigger DL RRC MESSAGE TRANSFER and confirm 'not retrievable' is handled without crash."
    ],
    "risks": [
      "Remote crash/DoS of gNB-CU upon malformed DU response",
      "Unbounded outage if attacker can repeatedly trigger failure path"
    ],
    "next_steps": [
      "Implement CU_handle_UE_CONTEXT_MODIFICATION_FAILURE with full parsing and graceful error handling per TS 38.473 §9.2.2.9.",
      "Add null/consistency checks before dereferencing IEs across all modification paths.",
      "Map ID inconsistencies to appropriate Cause and send Error Indication where applicable.",
      "Add unit and integration tests for UE Context Modification FAILURE/REFUSE and REQUIRED flows.",
      "Fuzz ASN.1 decode of related messages (including optional C-RNTI) to harden against malformed inputs."
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.